# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive import BanditPolicy
from azureml.train.hyperdrive import RandomParameterSampling
from azureml.train.hyperdrive import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import normal, uniform, choice
from azureml.core import ScriptRunConfig
from azureml.core import Environment
from azureml.core import Workspace, Experiment, RunConfiguration

from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AciWebservice
from azureml.core.model import Model

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
import os

import requests
import json

import joblib


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
workspace = Workspace.from_config()
experiment_name = 'hd-stroke-experiment'
experiment=Experiment(workspace, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [3]:
# Create compute cluster
cluster_name = "cpu-cluster"
try:
    compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
# Specify a Policy
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.01, delay_evaluation=5)

# Specify parameter sampler
param_sampling = RandomParameterSampling({"C": uniform(0.1, 1.0), 
                                 "max_iter": choice(50,100,150,200,250)})


env_name = 'AzureML-Tutorial'
environment = Environment.get(workspace=workspace, name=env_name)
script_run_config = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=environment
                      )


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_run_config = HyperDriveConfig(
                             run_config=script_run_config,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name='Accuracy',
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=15,
                             max_concurrent_runs=5)

In [5]:
experiment = Experiment(workspace, experiment_name)
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details



TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_ed911112-e614-4355-8c78-909832f684be
Web View: https://ml.azure.com/experiments/hd-stroke-experiment/runs/HD_ed911112-e614-4355-8c78-909832f684be?wsid=/subscriptions/12cd141f-7e8d-40f4-ba6e-35b327775bd4/resourcegroups/ska_resources/workspaces/capstone_project_workspace

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-03-30T10:18:19.267332][API][INFO]Experiment created<END>\n""<START>[2021-03-30T10:18:20.396726][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space<END>\n""<START>[2021-03-30T10:18:20.828279][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-03-30T10:18:21.6818136Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [14]:
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_hyperdrive_run)

best_run_metrics = best_hyperdrive_run.get_metrics()
print(best_run_metrics)

print('RunID: ', best_hyperdrive_run.id)

Run(Experiment: hd-stroke-experiment,
Id: HD_ed911112-e614-4355-8c78-909832f684be_1,
Type: azureml.scriptrun,
Status: Completed)
{'Regularization Strength:': 0.43836314014919653, 'Max iterations:': 200, 'Accuracy': 0.9549902152641878}
RunID:  HD_ed911112-e614-4355-8c78-909832f684be_1


In [15]:
#Save the best model
best_model = Model.register(workspace=workspace, 
                                model_path ="outputs/hyper-drive-model.pkl",
                              model_name = "hyper-drive-model",
                              tags = {"version": "1"},
                              description = "Stroke classification",
                          )

print(best_model)

Registering model hyper-drive-model
Model(workspace=Workspace.create(name='capstone_project_workspace', subscription_id='12cd141f-7e8d-40f4-ba6e-35b327775bd4', resource_group='ska_resources'), name=hyper-drive-model, id=hyper-drive-model:11, version=11, tags={'version': '1'}, properties={})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
env = Environment.get(workspace, "AzureML-Tutorial").clone(env_name)

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script="score.py", environment = env)

In [21]:
# Remote service
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1.8, memory_gb = 4,enable_app_insights = True)

service = Model.deploy(workspace=workspace,
                    name="stroke-service-endpoint",
                    models=[best_model],
                    inference_config=inference_config,
                    deployment_config=deployment_config,
                    overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-03-30 11:24:32+00:00 Creating Container Registry if not exists.
2021-03-30 11:24:32+00:00 Registering the environment.
2021-03-30 11:24:34+00:00 Use the existing image.
2021-03-30 11:24:34+00:00 Generating deployment configuration.
2021-03-30 11:24:37+00:00 Submitting deployment to compute..
2021-03-30 11:24:43+00:00 Checking the status of deployment stroke-service-endpoint..
2021-03-30 11:29:54+00:00 Checking the status of inference endpoint stroke-service-endpoint.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [22]:
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Healthy
http://8f7907ab-7430-4ab5-8ea1-dbdff32d0b00.eastus.azurecontainer.io/score
http://8f7907ab-7430-4ab5-8ea1-dbdff32d0b00.eastus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [23]:
# Test Remote service
data = {
    "data":
    [
        {
            'id': "0",
            'gender': "1",
            'age': "80",
            'hypertension': "1",
            'heart_disease': "0",
            'ever_married': False,
            'work_type': "0",
            'Residence_type': "0",
            'avg_glucose_level': "0",
            'bmi': "0",
            'smoking_status': "0",
        },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(service.scoring_uri, input_data, headers=headers)
print(resp.json())

{"result": [1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.get_logs()

# Delete all resources not used
service.delete()
compute_target.delete()
